### 質問セットからSPARQLを生成して結果を得て評価までの一連の流れのデモ
.envのOPENAI_API_KEY, PATH_DIR(このディレクトリへのパス)を指定してください


### 実験パラメタ
- データベース
- プロンプト
- 質問難易度
- 言語モデル(今後実装)

In [ ]:
import os
import json
import time
from dotenv import load_dotenv
from functions.SPARQL_executer import execute_query
from functions.SPARQL_generator import sparql_gen
from functions.prompt_maker import make_prompt
from functions.results_evaluater import evaluate_jaccard

load_dotenv()
db = 'rhea'

prompt_id = None
prompt_variable_id = None

if db == 'uniprot':
    prompt_id = 2
    prompt_variable_id = 9
elif db == 'rhea':
    prompt_id = 5
    prompt_variable_id = 10
elif db == 'bgee':
    prompt_id = 6
    prompt_variable_id = 11
elif db == 'uniprot_and_bgee':
    prompt_id = 7
    prompt_variable_id = 13
level = "EASY" # EASY, MEDIUM, HARD
verbose = True # 出力が出るようにする

In [ ]:
with open(f"questions/json_format/{db}.json", 'r') as f:
	questions = json.load(f)

In [ ]:
# save path
save_path = f'data/questions/easy_question_augmented_{db}_model_yaml.json'
save_path_with_results = f'data/questions/easy_question_augmented_with_results_{db}_model_yaml.json'

In [ ]:
endpoint = os.environ[f"ENDPOINT_{db.upper()}"]

In [ ]:
query_results = []
for question in questions:
    time.sleep(0.5)
    query_results += [execute_query(question, endpoint, "sparql", 10000, "")]
    print(question["id"], len(query_results[-1][0]))

# 各質問に対する結果を追加
for result, question_id in query_results:
    # 質問を見つけて結果を追加
    for question in questions:
        if question["id"] == question_id:
            question["results"] = result


Q1-1-1 1
Q1-2-1 1
Q1-3-1 1
Q1-4-1 1
Q1-5-1 1
Q1-6-1 1
Q2-1-1 3
Q2-2-1 3
Q2-3-1 3
Q2-4-1 3
Q2-5-1 3
Q2-6-1 3
Q3-1-1 3
Q3-2-1 3
Q3-3-1 3
Q3-4-1 3
Q3-5-1 3
Q3-6-1 3
Q3-1-2 9
Q3-2-2 9
Q3-3-2 9
Q3-4-2 9
Q3-5-2 9
Q3-6-2 9
Q4-1-1 1
Q4-2-1 1
Q4-3-1 1
Q4-4-1 1
Q4-5-1 1
Q4-6-1 1
Q5-1-1 1
Q5-2-1 1
Q5-3-1 1
Q5-4-1 1
Q5-5-1 1
Q5-6-1 1
Q6-1-1 1
Q6-2-1 1
Q6-3-1 1
Q6-4-1 1
Q6-5-1 1
Q6-6-1 1
Q7-1-1 1
Q7-2-1 1
Q7-3-1 1
Q7-4-1 1
Q7-5-1 1
Q7-6-1 1
Q8-1-1 1
Q8-2-1 1
Q8-3-1 1
Q8-4-1 1
Q8-5-1 1
Q8-6-1 1
Q9-1-1 20
Q9-2-1 20
Q9-3-1 20
Q9-4-1 20
Q9-5-1 20
Q9-6-1 20
Q10-1-1 1
Q10-2-1 1
Q10-3-1 1
Q10-4-1 1
Q10-5-1 1
Q10-6-1 1
Q11-1-1 10
Q11-2-1 10
Q11-3-1 10
Q11-4-1 10
Q11-5-1 10
Q11-6-1 10
Q12-1-1 5
Q12-2-1 5
Q12-3-1 5
Q12-4-1 5
Q12-5-1 5
Q12-6-1 5
Q13-1-1 1
Q13-2-1 1
Q13-3-1 1
Q13-4-1 1
Q13-5-1 1
Q13-6-1 1
Q14-1-1 1
Q14-2-1 1
Q14-3-1 1
Q14-4-1 1
Q14-5-1 1
Q14-6-1 1
Q15-1-1 2
Q15-2-1 2
Q15-3-1 2
Q15-4-1 2
Q15-5-1 2
Q15-6-1 2
Q16-1-1 3
Q16-2-1 3
Q16-3-1 3
Q16-4-1 3
Q16-5-1 3
Q16-6-1 3
Q16-1-2 9
Q16-2-2 9
Q16-3-2 

In [ ]:
with open(save_path_with_results, 'w') as f:
    json.dump(questions, f, indent=4)

In [9]:
prompt_variable_id

10

In [11]:
# llm入力用のプロンプトを作成
questions = make_prompt(db, prompt_id, prompt_variable_id, questions)

In [14]:
# llmに作成したプロンプトを入力して出力からRDF-configを利用してSPARQLを生成
questions = sparql_gen(db, questions, verbose)

############################################################################################################################################################################################################################################################################################################
llm_output: 1. Variables
What does user look for:
- reaction formula
variables to look for based on elements in [variables_info]:
- reaction_equation

2. Conditions
conditions to narrow down:
- rhea:10024
variables to narrow down based on elements in [variables_info]. upper variables and conditions should be connected entity like person_ and Person :
- Reaction
condition and variable (If it's a name, use a full name with underscore URI and prefix res:) pair:
- {Reaction: rhea:10024}
------------------------------
Variables: ['reaction_equation']
------------------------------
Parameters: {'Reaction': 'rhea:10024'}
#############################################################################

In [15]:
# データベースのエンドポイントを取得
endpoint = os.environ[f"ENDPOINT_{db.upper()}"]

In [ ]:
# エンドポイントにSPARQLを投げて結果を取得
query_results = []
for question in questions:
    query_results += [execute_query(question, endpoint, "llm_rdf_result", 10000, "")]
    print(question["id"], len(query_results[-1][0]))

# 各質問に対する結果を追加
for result, question_id in query_results:
    # 質問を見つけて結果を追加
    for question in questions:
        if question["id"] == question_id:
            question["results"] = result

Q1-1-1 1
Q1-2-1 1
Q1-3-1 1
Q1-4-1 1
Q1-5-1 1
Q1-6-1 1
Q2-1-1 3
Execute Error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'<!DOCTYPE html SYSTEM "about:legacy-compat">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>Rhea</title><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><link href="/" rel="home"/><link href="/base.css" type="text/css" rel="stylesheet"/><link type="image/vnd.microsoft.icon" href="https://www.rhea-db.org//favicon.ico" rel="shortcut icon"/><link href="/rhea-sparql.css" type="text/css" rel="stylesheet"/><link href="https://creativecommons.org/licenses/by/4.0/" rel="license"/><script src="https://cdn.jsdelivr.net/npm/yasgui-yasqe@2.11.22/dist/yasqe.bundled.min.js"></script><link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/yasqe/2.2/yasqe.min.css"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta cont

In [17]:
# save questions
with open(save_path, "w") as f:
    json.dump(questions, f, indent=2)

回答と比較して評価をする

In [7]:
import json
level = "EASY"
db = 'uniprot_and_bgee'

from functions.results_evaluater import evaluate_jaccard

In [8]:
# load 
save_path = f'data/questions/easy_question_augmented_{db}_model_yaml.json'
save_path_with_results = f'data/questions/easy_question_augmented_with_results_{db}_model_yaml.json'

with open(save_path) as f:
    questions = json.load(f)
with open(save_path_with_results) as f:
    answers = json.load(f)

In [9]:
len(questions), len(answers)

(102, 102)

In [10]:
# 正解出力との比較を行う
score = evaluate_jaccard(questions, answers)

Evaluating:   6%|▌         | 6/102 [00:03<00:28,  3.42it/s]

Skipping Q2-1-1 due to missing results.


Evaluating:  11%|█         | 11/102 [00:04<00:23,  3.89it/s]

Skipping Q2-6-1 due to missing results.
Skipping Q3-1-1 due to empty columns.
Skipping Q3-2-1 due to empty columns.
Skipping Q3-3-1 due to empty columns.
Skipping Q3-4-1 due to empty columns.
Skipping Q3-5-1 due to missing results.
Skipping Q3-6-1 due to missing results.


Evaluating:  30%|███       | 31/102 [00:07<00:14,  5.03it/s]

...

Skipping Q8-1-1 due to empty columns.
Skipping Q8-2-1 due to missing results.
Skipping Q8-3-1 due to empty columns.
Skipping Q8-4-1 due to empty columns.
Skipping Q8-5-1 due to empty columns.
Skipping Q8-6-1 due to empty columns.
Skipping Q9-1-1 due to empty columns.
Skipping Q9-2-1 due to empty columns.
Skipping Q9-3-1 due to empty columns.
Skipping Q9-5-1 due to missing results.


Evaluating:  64%|██████▎   | 65/102 [00:20<00:08,  4.24it/s]

Skipping Q10-1-1 due to empty columns.
Skipping Q10-2-1 due to missing results.
Skipping Q10-3-1 due to empty columns.
Skipping Q10-4-1 due to empty columns.
Skipping Q10-5-1 due to empty columns.
Skipping Q10-6-1 due to empty columns.
Skipping Q11-1-1 due to empty columns.
Skipping Q11-2-1 due to empty columns.
Skipping Q11-3-1 due to empty columns.
Skipping Q11-4-1 due to empty columns.
Skipping Q11-5-1 due to empty columns.
Skipping Q11-6-1 due to empty columns.
Skipping Q12-1-1 due to empty columns.
Skipping Q12-2-1 due to empty columns.
Skipping Q12-3-1 due to empty columns.
Skipping Q12-4-1 due to missing results.
Skipping Q12-5-1 due to empty columns.
Skipping Q12-6-1 due to empty columns.
Skipping Q13-4-1 due to empty columns.
Skipping Q13-6-1 due to missing results.


Evaluating: 100%|██████████| 102/102 [00:20<00:00,  4.93it/s]

Skipping Q15-2-1 due to missing results.
Skipping Q15-3-1 due to missing results.
Skipping Q15-4-1 due to empty columns.
Skipping Q15-5-1 due to missing results.
Skipping Q15-6-1 due to missing results.
Skipping Q16-1-1 due to empty columns.
Skipping Q16-2-1 due to empty columns.
Skipping Q16-3-1 due to empty columns.
Skipping Q16-4-1 due to empty columns.
Skipping Q16-5-1 due to empty columns.
Skipping Q16-6-1 due to empty columns.
Skipping Q17-2-1 due to missing results.
Skipping Q17-5-1 due to missing results.
Skipping Q17-6-1 due to missing results.


In [12]:
score

{'Q1-1-1': {'jaccard_score': 0.9999},
 'Q1-2-1': {'jaccard_score': 0.9999},
 'Q1-3-1': {'jaccard_score': 0.9999},
 'Q1-4-1': {'jaccard_score': 0.9999},
 'Q1-5-1': {'jaccard_score': 0.9999},
 'Q1-6-1': {'jaccard_score': 0.9999},
 'Q2-1-1': {'jaccard_score': 0},
 'Q2-2-1': {'jaccard_score': 1.0},
 'Q2-3-1': {'jaccard_score': 1.0},
 'Q2-4-1': {'jaccard_score': 1.0},
 'Q2-5-1': {'jaccard_score': 0.0},
 'Q2-6-1': {'jaccard_score': 0},
 'Q3-1-1': {'jaccard_score': 0},
 'Q3-2-1': {'jaccard_score': 0},
 'Q3-3-1': {'jaccard_score': 0},
 'Q3-4-1': {'jaccard_score': 0},
 'Q3-5-1': {'jaccard_score': 0},
 'Q3-6-1': {'jaccard_score': 0},
 'Q4-1-1': {'jaccard_score': 0.0},
 'Q4-2-1': {'jaccard_score': 0.0},
 'Q4-3-1': {'jaccard_score': 0.0},
 'Q4-4-1': {'jaccard_score': 0.0},
 'Q4-5-1': {'jaccard_score': 0.0},
 'Q4-6-1': {'jaccard_score': 0.0},
 'Q5-1-1': {'jaccard_score': 0.0283},
 'Q5-2-1': {'jaccard_score': 0.9988221436984688},
 'Q5-3-1': {'jaccard_score': 0.0283},
 'Q5-4-1': {'jaccard_score': 0.0